In [ ]:
import requests
import os
import json
import re

# --- CONFIGURATION ---
# !!! IMPORTANT: SET YOUR OBSIDIAN SNIPPET PATH HERE !!!
SNIPPET_DIR = r"C:\Users\BalasubramanianPG\Videos\Obsidian Vault\.obsidian\snippets"

# The source for Simple Icons data
SIMPLE_ICONS_URL = "https://simpleicons.org/"

# Target fill color for the SVG icon path (You wanted WHITE)
ICON_FILL_COLOR = "#FFFFFF"

# ----------------------------------------------------
# --- HELPER FUNCTIONS ---
# ----------------------------------------------------

def slugify(name):
    """Converts a name like 'Visual Studio Code' to 'visual-studio-code'"""
    # This matches the common slugification style for Simple Icons
    name = re.sub(r'[^a-zA-Z0-9\s]+', '', name).strip()
    return name.lower().replace(' ', '-')

def encode_svg(svg_path_data, fill_color):
    """
    Creates a URL-encoded SVG string suitable for CSS background-image.
    This is critical for the icon to display correctly.
    """
    # Create the raw SVG structure
    svg_raw = f'<svg role="img" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg"><title>Icon</title><path fill="{fill_color}" d="{svg_path_data}"/></svg>'
    
    # URL-encode the necessary characters
    svg_encoded = svg_raw.replace('"', "'")\
                         .replace('<', '%3c')\
                         .replace('>', '%3e')\
                         .replace('#', '%23')
    
    # The final data URL string
    return f"url('data:image/svg+xml;utf8,{svg_encoded}')"

def hex_to_rgb(hex_color):
    """Converts hex color (e.g., '123456') to 'R, G, B' string for --callout-color"""
    hex_color = hex_color.lstrip('#')
    # Convert from hex string to integers (0-255)
    r = int(hex_color[0:2], 16)
    g = int(hex_color[2:4], 16)
    b = int(hex_color[4:6], 16)
    return f"{r}, {g}, {b}"


# ----------------------------------------------------
# --- MAIN EXECUTION ---
# ----------------------------------------------------

def generate_callout_snippets():
    """Fetches data and generates all CSS snippet files."""
    
    print(f"Starting script. Target directory: {SNIPPET_DIR}")

    # 1. Ensure the directory exists
    os.makedirs(SNIPPET_DIR, exist_ok=True)
    print("Snippet directory ensured.")
    
    # 2. Fetch the Simple Icons data
    try:
        print("Fetching Simple Icons data...")
        response = requests.get(SIMPLE_ICONS_URL)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        data = response.json()
        icons = data.get('icons', [])
        print(f"Successfully retrieved data for {len(icons)} icons.")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching icon data. Please check your internet connection: {e}")
        return

    # 3. Process and write each icon
    count = 0
    
    for icon in icons:
        try:
            name = icon['title']
            slug = slugify(name)
            hex_color = icon['hex']
            svg_path_data = icon['path']
            
            # Obsidian Variables
            callout_keyword = slug
            callout_rgb = hex_to_rgb(hex_color)
            encoded_svg = encode_svg(svg_path_data, ICON_FILL_COLOR)
            
            # The CSS Template
            css_content = f"""/* --- Custom Callout: {name} ({hex_color}) --- */
.callout[data-callout='{callout_keyword}'] {{
  /* Callout box background color (primary brand color) */
  --callout-color: {callout_rgb};

  /* Placeholder icon (required, but hidden below) */
  --callout-icon: lucide-package; 
}}

/* Custom SVG Injection */
.callout[data-callout='{callout_keyword}'] .callout-icon {{
  visibility: hidden; 
  position: relative;
  width: 1em; /* Ensure it takes up space */
  height: 1em;
}}

.callout[data-callout='{callout_keyword}'] .callout-icon:after {{
  content: '';
  visibility: visible;
  position: absolute;
  top: 0;
  left: 0;
  width: 100%;
  height: 100%;
  
  /* ENCODED SVG CODE */
  background-image: {encoded_svg};
  
  /* Styling to make the SVG fit */
  background-size: contain;
  background-repeat: no-repeat;
  background-position: center;
}}
"""
            # Write the file
            file_name = f"{slug}.css"
            file_path = os.path.join(SNIPPET_DIR, file_name)
            
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(css_content)
                
            count += 1

        except Exception as e:
            print(f"Failed to process icon '{name}'. Error: {e}")
            continue

    print(f"\n--- DONE! ---")
    print(f"Successfully created {count} custom callout CSS files.")
    print("Remember to restart Obsidian or reload CSS snippets in settings.")


if __name__ == "__main__":
    generate_callout_snippets()

In [ ]:
import os

vault_path = r"C:\Users\BalasubramanianPG\Videos\Obsidian Vault\Pharma Domain\The Training Document"

markdown_files = []

for root, dirs, files in os.walk(vault_path):
    for file in files:
        if file.lower().endswith(".md"):
            full_path = os.path.join(root, file)
            markdown_files.append(full_path)

for path in markdown_files:
    print(path)


In [7]:
import os
from pathlib import Path
from datetime import date
from typing import Dict, List, Any

# --- CONFIGURATION SECTION ---
# Define the root path of your Obsidian vault
VAULT_ROOT_PATH = Path(r"C:\Users\BalasubramanianPG\Videos\Obsidian Vault\Snowflake")

# Define the structure and initial content for your markdown files.
# Key: Folder Name
# Value: Dictionary where
#   Key: Subfolder Name
#   Value: List of Markdown filenames (without .md extension)
VAULT_STRUCTURE: Dict[str, Dict[str, List[str]]] = {
    "Architecture": {
        "Cloud": ["Overview", "Security", "Networking"],
        "Components": ["Virtual Warehouse", "Storage", "Cloud Services"]
    },
    "SQL": {
        "DML": ["Insert", "Update", "Delete"],
        "DDL": ["Create Table", "Alter Table", "Drop Table"]
    },
    "Data Loading": {
        "Internal": ["Staging", "Copy Command"],
        "External": ["S3 Integration", "Azure Blob", "GCS Bucket"]
    },
    "Best Practices": {
        "Performance": ["Clustering", "Micro-partitions", "Query Tuning"],
        "Cost Management": ["Warehouse Sizing", "Resource Monitors"]
    }
}

# --- YAML FRONT MATTER TEMPLATE FUNCTION ---

def create_yaml_front_matter(area: str, sub_area: str, filename: str) -> str:
    """
    Generates the YAML front matter string for an Obsidian markdown file.
    
    The 'Area' property is formatted as an Obsidian internal link for easy navigation.
    The 'Tag' property is a multi-list, which is standard YAML/Obsidian practice.
    """
    today = date.today().isoformat()
    
    # Clean up names for Description and Sub Area
    description = f"Notes on the {sub_area} aspect of {area}, focusing on {filename} in Snowflake."
    
    # Format the Area as an Obsidian link, e.g., '[[Architecture/Cloud]]'
    obsidian_link_area = f"[[{area}/{sub_area}]]"
    
    # Generate relevant tags
    tags_list = [
        "Snowflake", 
        area.replace(" ", "-"), 
        sub_area.replace(" ", "-"), 
        filename.replace(" ", "-")
    ]
    
    yaml_template = f"""---
Created Date: {today}
Area: {obsidian_link_area}
Description: "{description}"
Sub area: "{sub_area} / {filename}"
Tag:
  - {tags_list[0]}
  - {tags_list[1]}
  - {tags_list[2]}
  - {tags_list[3]}
---

# {area} / {sub_area} / {filename}

## Overview

[Start writing your notes here...]
"""
    return yaml_template

# --- MAIN SCRIPT EXECUTION ---

def create_vault_structure(root_path: Path, structure_map: Dict[str, Dict[str, List[str]]]):
    """
    Creates the folders, subfolders, and markdown files based on the structure map.
    """
    print(f"Starting creation process in: {root_path}")
    
    if not root_path.exists():
        print(f"Base path '{root_path}' does not exist. Creating it.")
        root_path.mkdir(parents=True, exist_ok=True)
    
    total_files = 0

    for folder_name, subfolder_map in structure_map.items():
        # Create the main folder
        main_folder_path = root_path / folder_name
        main_folder_path.mkdir(exist_ok=True)
        print(f"  Created Folder: {folder_name}")

        for subfolder_name, filenames in subfolder_map.items():
            # Create the subfolder
            subfolder_path = main_folder_path / subfolder_name
            subfolder_path.mkdir(exist_ok=True)
            print(f"    Created Subfolder: {folder_name}/{subfolder_name}")

            for filename in filenames:
                # Create the full file path
                file_path = subfolder_path / f"{filename}.md"
                
                # Generate the content with YAML front matter
                file_content = create_yaml_front_matter(folder_name, subfolder_name, filename)
                
                # Write the content to the file
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(file_content)
                
                total_files += 1
                print(f"      Created File: {filename}.md")

    print(f"\n✅ SUCCESS! {total_files} files and the full directory structure have been created.")
    print("Your Obsidian Vault is ready!")

if __name__ == "__main__":
    create_vault_structure(VAULT_ROOT_PATH, VAULT_STRUCTURE)

Starting creation process in: C:\Users\BalasubramanianPG\Videos\Obsidian Vault\Snowflake
  Created Folder: Architecture
    Created Subfolder: Architecture/Cloud
      Created File: Overview.md
      Created File: Security.md
      Created File: Networking.md
    Created Subfolder: Architecture/Components
      Created File: Virtual Warehouse.md
      Created File: Storage.md
      Created File: Cloud Services.md
  Created Folder: SQL
    Created Subfolder: SQL/DML
      Created File: Insert.md
      Created File: Update.md
      Created File: Delete.md
    Created Subfolder: SQL/DDL
      Created File: Create Table.md
      Created File: Alter Table.md
      Created File: Drop Table.md
  Created Folder: Data Loading
    Created Subfolder: Data Loading/Internal
      Created File: Staging.md
      Created File: Copy Command.md
    Created Subfolder: Data Loading/External
      Created File: S3 Integration.md
      Created File: Azure Blob.md
      Created File: GCS Bucket.md
  Created 